In [13]:
import os
import re
import io
import csv
import time
import json
import js2py
import ctypes
import random
import loguru
import pyquery
import chardet
import datetime
import fractions
import urllib.parse
import requests, requests.exceptions
import camelot, camelot.ext.ghostscript._gsprint


## 從證交所取得資料並存在Propotion

In [3]:
class Propotion:
    # 建構子
    def __init__(self, sort, code, name, percent):
        # 排序
        # 轉換為整數，以利後續排序使用
        self.Sort = int(sort)
        # 證券代碼
        self.Code = code
        # 證券名稱
        self.Name = name
        # 市值佔比
        # 不建議使用 float 進行浮點數保存，後續可能會造成浮點數計算誤差，
        # 使用 Fraction 保存才能有效降低浮點數計算結果的誤差
        self.Percent = fractions.Fraction(percent[:-1])
    # 物件表達式
    def __repr__(self):
        return (
            f'class Propotion {{ '
            f'Sort={self.Sort}, '
            f'Code={self.Code}, '
            f'Name={self.Name}, '
            f'Percent={float(self.Percent):.4f}% '
            f'}}'
        )

In [4]:
def main():
    resp = requests.get('https://www.taifex.com.tw/cht/9/futuresQADetail')
    if resp.status_code != 200:
        loguru.logger.error('RESP: status code is not 200')
    loguru.logger.success('RESP: success')

    txt = None
    det = chardet.detect(resp.content)
    try:
        if det['confidence'] > 0.5:
            if det['encoding'] == 'big-5':
                txt = resp.content.decode('big5')
            else:
                txt = resp.content.decode(det['encoding'])
        else:
            txt = resp.content.decode('utf-8')
    except Exception as e:
        loguru.logger.error(e)

    if txt is None:
        return
    loguru.logger.info(txt)

    proportions = []

    d = pyquery.PyQuery(txt)
    trs = list(d('table tr').items())
    trs = trs[1:]
    for tr in trs:
        tds = list(tr('td').items())
        code = tds[1].text().strip()
        if code != '':
            sort = tds[1].text().strip()
            name = tds[2].text().strip()
            percent = tds[3].text().strip()
            proportions.append(Propotion(
                sort=sort,
                code=code,
                name=name,
                percent=percent
            ))
        code = tds[5].text().strip()
        if code != '':
            sort = tds[5].text().strip()
            name = tds[6].text().strip()
            percent = tds[7].text().strip()
            proportions.append(Propotion(
                sort=sort,
                code=code,
                name=name,
                percent=percent
            ))

    proportions.sort(key=lambda proportion: proportion.Code)
    loguru.logger.info(proportions)

    # 將每筆物件表達式輸出的字串以系統換行符號相接，讓每筆物件表達式各自獨立一行
    message = os.linesep.join([str(proportion) for proportion in proportions])
    loguru.logger.info('PROPORTIONS' + os.linesep + message)

if __name__ == '__main__':
    loguru.logger.add(
        f'{datetime.date.today():%Y%m%d}.log',
        rotation='1 day',
        retention='7 days',
        level='DEBUG'
    )
    main()

2020-04-29 16:50:28.323 | SUCCESS  | __main__:main:5 - RESP: success
2020-04-29 16:50:28.989 | INFO     | __main__:main:22 - <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="zh-TW">

<head><meta http-equiv="X-UA-Compatible" content="IE=edge"></head><head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  <link rel="shortcut icon" type="image/x-icon" href="/cht/resources/front/cht/images/favicon.ico" />
  <meta property="og:image" content="https://www.taifex.com.tw/chinese/images/fb_logo.jpg" />
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
  <title>臺灣證券交易所發行量加權股價指數成分股暨市值比重</title>
  <link rel="stylesheet" type="text/css" href="/cht/resources/front/cht/css/global.css" />
  <link rel="stylesheet" type="text/css" href="/cht/resources/front/cht/css/content.css" />
  <script src="/cht/resources/front/js/jquery-3.1.1.min.js" type="text/javascript"></script>

2020-04-29 16:50:29.128 | INFO     | __main__:main:55 - [class Propotion { Sort=1101, Code=1101, Name=台泥, Percent=0.7390% }, class Propotion { Sort=1102, Code=1102, Name=亞泥, Percent=0.4534% }, class Propotion { Sort=1103, Code=1103, Name=嘉泥, Percent=0.0402% }, class Propotion { Sort=1104, Code=1104, Name=環泥, Percent=0.0354% }, class Propotion { Sort=1108, Code=1108, Name=幸福, Percent=0.0087% }, class Propotion { Sort=1109, Code=1109, Name=信大, Percent=0.0187% }, class Propotion { Sort=1110, Code=1110, Name=東泥, Percent=0.0315% }, class Propotion { Sort=1201, Code=1201, Name=味全, Percent=0.0346% }, class Propotion { Sort=1203, Code=1203, Name=味王, Percent=0.0215% }, class Propotion { Sort=1210, Code=1210, Name=大成, Percent=0.1046% }, class Propotion { Sort=1215, Code=1215, Name=卜蜂, Percent=0.0542% }, class Propotion { Sort=1216, Code=1216, Name=統一, Percent=1.2759% }, class Propotion { Sort=1217, Code=1217, Name=愛之味, Percent=0.0116% }, class Propotion { Sort=1218, Code=1218, Name=泰山, Percent=0

2020-04-29 16:50:29.135 | INFO     | __main__:main:59 - PROPORTIONS
class Propotion { Sort=1101, Code=1101, Name=台泥, Percent=0.7390% }
class Propotion { Sort=1102, Code=1102, Name=亞泥, Percent=0.4534% }
class Propotion { Sort=1103, Code=1103, Name=嘉泥, Percent=0.0402% }
class Propotion { Sort=1104, Code=1104, Name=環泥, Percent=0.0354% }
class Propotion { Sort=1108, Code=1108, Name=幸福, Percent=0.0087% }
class Propotion { Sort=1109, Code=1109, Name=信大, Percent=0.0187% }
class Propotion { Sort=1110, Code=1110, Name=東泥, Percent=0.0315% }
class Propotion { Sort=1201, Code=1201, Name=味全, Percent=0.0346% }
class Propotion { Sort=1203, Code=1203, Name=味王, Percent=0.0215% }
class Propotion { Sort=1210, Code=1210, Name=大成, Percent=0.1046% }
class Propotion { Sort=1215, Code=1215, Name=卜蜂, Percent=0.0542% }
class Propotion { Sort=1216, Code=1216, Name=統一, Percent=1.2759% }
class Propotion { Sort=1217, Code=1217, Name=愛之味, Percent=0.0116% }
class Propotion { Sort=1218, Code=1218, Name=泰山, Percent=0.0

## 取得可免費使用之IP代理 避免因頻繁存取而被封鎖

In [14]:
def getProxiesFromProxyNova():
    proxies = []
    # 按照網站規則使用各國代碼傳入網址取得各國 IP 代理
    countries = [
        'tw',
        'jp',
        'kr',
        'id',
        'my',
        'th',
        'vn',
        'ph',
        'hk',
        'uk',
        'us'
    ]
    for country in countries:
        url = f'https://www.proxynova.com/proxy-server-list/country-{country}/'
        loguru.logger.debug(f'getProxiesFromProxyNova: {url}')
        loguru.logger.warning(f'getProxiesFromProxyNova: downloading...')
        response = requests.get(url, verify=False)
        if response.status_code != 200:
            loguru.logger.debug(f'getProxiesFromProxyNova: status code is not 200')
            continue
        loguru.logger.success(f'getProxiesFromProxyNova: downloaded.')
        d = pyquery.PyQuery(response.text)
        table = d('table#tbl_proxy_list')
        rows = list(table('tbody:first > tr').items())
        loguru.logger.warning(f'getProxiesFromProxyNova: scanning...')
        for row in rows:
            tds = list(row('td').items())
            # 若為分隔行則僅有 1 格
            if len(tds) == 1:
                continue
            # 取出 IP 欄位內的 JavaScript 程式碼
            js = row('td:nth-child(1) > abbr').text()
            # 去除 JavaScript 程式碼開頭的 document.write( 字串與結尾的 ); 字串，
            # 再與可供 js2py 執行後回傳指定變數的 JavaScript 程式碼相結合
            js = 'let x = %s; x' % (js[15:-2])
            # 透過 js2py 執行取得還原後的 IP
            ip = js2py.eval_js(js).strip()
            # 取出 Port 欄位值
            port = row('td:nth-child(2)').text().strip()
            # 組合 IP 代理
            proxy = f'{ip}:{port}'
            proxies.append(proxy)
        loguru.logger.success(f'getProxiesFromProxyNova: scanned.')
        loguru.logger.debug(f'getProxiesFromProxyNova: {len(proxies)} proxies is found.')
        # 每取得一個國家代理清單就休息一秒，避免頻繁存取導致代理清單網站封鎖
        time.sleep(1)
    return proxies

In [15]:
def getProxiesFromFreeProxyList(): #work
    proxies = []
    url = 'https://free-proxy-list.net/'
    loguru.logger.debug(f'getProxiesFromFreeProxyList: {url}')
    loguru.logger.warning(f'getProxiesFromFreeProxyList: downloading...')
    response = requests.get(url, verify=False)
    if response.status_code != 200:
        loguru.logger.debug(f'getProxiesFromFreeProxyList: status code is not 200')
        return
    loguru.logger.success(f'getProxiesFromFreeProxyList: downloaded.')
    d = pyquery.PyQuery(response.text)
    trs = list(d('table#proxylisttable > tbody > tr').items())
    loguru.logger.warning(f'getProxiesFromFreeProxyList: scanning...')
    for tr in trs:
        # 取出所有資料格
        tds = list(tr('td').items())
        # 取出 IP 欄位值
        ip = tds[0].text().strip()
        # 取出 Port 欄位值
        port = tds[1].text().strip()
        # 組合 IP 代理
        proxy = f'{ip}:{port}'
        proxies.append(proxy)
    loguru.logger.success(f'getProxiesFromFreeProxyList: scanned.')
    loguru.logger.debug(f'getProxiesFromFreeProxyList: {len(proxies)} proxies is found.')
    return proxies

In [16]:
now = datetime.datetime.now()
proxies = []

# 隨機取出一組代理
def getProxy():
    global proxies
    # 若代理清單內已無代理，則重新下載
    if len(proxies) == 0:
        getProxies()
    proxy = random.choice(proxies)
    loguru.logger.debug(f'getProxy: {proxy}')
    proxies.remove(proxy)
    loguru.logger.debug(f'getProxy: {len(proxies)} proxies is unused.')
    return proxy

def reqProxies(hour):
    global proxies
    proxies = proxies + getProxiesFromProxyNova()
#     proxies = proxies + getProxiesFromGatherProxy() 壞了
    proxies = proxies + getProxiesFromFreeProxyList()
    proxies = list(dict.fromkeys(proxies))
    loguru.logger.debug(f'reqProxies: {len(proxies)} proxies is found.')

def getProxies():
    global proxies
    hour = f'{now:%Y%m%d%H}'
    filename = f'proxies-{hour}.csv'
    filepath = f'{filename}'
    if os.path.isfile(filepath):
        loguru.logger.info(f'getProxies: {filename} exists.')
        loguru.logger.warning(f'getProxies: {filename} is loading...')
        with open(filepath, 'r', newline='', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                proxy = row['Proxy']
                proxies.append(proxy)
        loguru.logger.success(f'getProxies: {filename} is loaded.')
    else:
        loguru.logger.info(f'getProxies: {filename} does not exist.')
        reqProxies(hour)
        loguru.logger.warning(f'getProxies: {filename} is saving...')
        with open(filepath, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([
                'Proxy'
            ])
            for proxy in proxies:
                writer.writerow([
                    proxy
                ])
        loguru.logger.success(f'getProxies: {filename} is saved.')

In [17]:
# 用於保存上一次連線請求成功時用的代理資訊
proxy = None

def testRequest():
    global proxy
    # 持續更換代理直到連線請求成功為止
    while True:
        # 若無上一次連線請求成功的代理資訊，則重新取出一組代理資訊
        if proxy is None:
            proxy = getProxy()
        try:
            url = f'https://www.google.com/'
            loguru.logger.info(f'testRequest: url is {url}')
            loguru.logger.warning(f'testRequest: downloading...')
            response = requests.get(
                url,
                # 指定 HTTPS 代理資訊
                proxies={
                    'https': f'https://{proxy}'
                },
                # 指定連限逾時限制
                timeout=5
            )
            if response.status_code != 200:
                loguru.logger.debug(f'testRequest: status code is not 200.')
                # 請求發生錯誤，清除代理資訊，繼續下個迴圈
                proxy = None
                continue
            loguru.logger.success(f'testRequest: downloaded.')
        # 發生以下各種例外時，清除代理資訊，繼續下個迴圈
        except requests.exceptions.ConnectionError:
            loguru.logger.error(f'testRequest: proxy({proxy}) is not working (connection error).')
            proxy = None
            continue
        except requests.exceptions.ConnectTimeout:
            loguru.logger.error(f'testRequest: proxy({proxy}) is not working (connect timeout).')
            proxy = None
            continue
        except requests.exceptions.ProxyError:
            loguru.logger.error(f'testRequest: proxy({proxy}) is not working (proxy error).')
            proxy = None
            continue
        except requests.exceptions.SSLError:
            loguru.logger.error(f'testRequest: proxy({proxy}) is not working (ssl error).')
            proxy = None
            continue
        except Exception as e:
            loguru.logger.error(f'testRequest: proxy({proxy}) is not working.')
            loguru.logger.error(e)
            proxy = None
            continue
        # 成功完成請求，離開迴圈
        break

In [18]:
testRequest()

2020-04-29 16:52:17.543 | INFO     | __main__:getProxies:30 - getProxies: proxies-2020042916.csv exists.
2020-04-29 16:52:17.544 | WARNING  | __main__:getProxies:31 - getProxies: proxies-2020042916.csv is loading...
2020-04-29 16:52:17.546 | SUCCESS  | __main__:getProxies:37 - getProxies: proxies-2020042916.csv is loaded.
2020-04-29 16:52:17.547 | DEBUG    | __main__:getProxy:11 - getProxy: 186.64.120.69:80
2020-04-29 16:52:17.548 | DEBUG    | __main__:getProxy:13 - getProxy: 498 proxies is unused.
2020-04-29 16:52:17.549 | INFO     | __main__:testRequest:13 - testRequest: url is https://www.google.com/
2020-04-29 16:52:17.549 | WARNING  | __main__:testRequest:14 - testRequest: downloading...
2020-04-29 16:52:17.715 | SUCCESS  | __main__:testRequest:29 - testRequest: downloaded.


## 取得個股盤後資訊

In [19]:
class AfterHoursInfo:
    def __init__(
        self,
        code,
        name,
        totalShare,
        totalTurnover,
        openPrice,
        highestPrice,
        lowestPrice,
        closePrice):
        # 代碼
        self.Code = code
        # 名稱
        self.Name = name
        # 成交股數
        self.TotalShare = self.checkNumber(totalShare)
        if self.TotalShare is not None:
            self.TotalShare = int(totalShare)
        # 成交金額
        self.TotalTurnover = self.checkNumber(totalTurnover)
        if self.TotalTurnover is not None:
            self.TotalTurnover = int(totalTurnover)
        # 開盤價
        self.OpenPrice = self.checkNumber(openPrice)
        if self.OpenPrice is not None:
            self.OpenPrice = fractions.Fraction(openPrice)
        # 最高價
        self.HighestPrice = self.checkNumber(highestPrice)
        if self.HighestPrice is not None:
            self.HighestPrice = fractions.Fraction(highestPrice)
        # 最低價
        self.LowestPrice = self.checkNumber(lowestPrice)
        if self.LowestPrice is not None:
            self.LowestPrice = fractions.Fraction(lowestPrice)
        # 收盤價
        self.ClosePrice = self.checkNumber(closePrice)
        if self.ClosePrice is not None:
            self.ClosePrice = fractions.Fraction(closePrice)
    # 物件表達式
    def __repr__(self):
        totalShare = self.TotalShare
        if totalShare is not None:
            totalShare = f'{totalShare}'
        totalTurnover = self.TotalTurnover
        if totalTurnover is not None:
            totalTurnover = f'{totalTurnover}'
        openPrice = self.OpenPrice
        if openPrice is not None:
            openPrice = f'{float(openPrice):.2f}'
        highestPrice = self.HighestPrice
        if highestPrice is not None:
            highestPrice = f'{float(highestPrice):.2f}'
        lowestPrice = self.LowestPrice
        if lowestPrice is not None:
            lowestPrice = f'{float(lowestPrice):.2f}'
        closePrice = self.ClosePrice
        if closePrice is not None:
            closePrice = f'{float(closePrice):.2f}'
        return (
            f'class AfterHoursInfo {{ '
            f'Code={self.Code}, '
            f'Name={self.Name}, '
            f'TotalShare={totalShare}, '
            f'TotalTurnover={totalTurnover}, '
            f'OpenPrice={openPrice}, '
            f'HighestPrice={highestPrice}, '
            f'LowestPrice={lowestPrice}, '
            f'ClosePrice={closePrice} '
            f'}}'
        )
    # 檢查數值是否有效
    def checkNumber(self, value):
        if value == '--':
            return None
        else:
            return value

In [20]:
def main():
    # 下載當日個股盤後資訊
    resp = requests.get(
        f'https://www.twse.com.tw/exchangeReport/MI_INDEX?' +
        f'response=json&' +
        f'type=ALLBUT0999' +
#         f'&date={datetime.date.today():%Y%m%d}'
        f'&date=20200424'
    )
    if resp.status_code != 200:
        loguru.logger.error('RESP: status code is not 200')
    loguru.logger.success('RESP: success')

    # 盤後資訊清單
    afterHoursInfos = []

    # 取出 JSON 內容
    body = resp.json()
    # 取出 stat 欄位
    stat = body['stat']
    # 如果 stat 不是 OK，代表查詢日期尚無資料
    if stat != 'OK':
        loguru.logger.error(f'RESP: body.stat error is {stat}.')
        return
    # 取出第 9 表格內容
    records = body['data9']
    # 依序取出每筆盤後資訊
    for record in records:
        # 取出代碼欄位值
        code = record[0].strip()
        # 符合股票代碼規則才處理
        if re.match(r'^[1-9][0-9][0-9][0-9]$', code) is not None:
            # 取出名稱欄位值
            name = record[1].strip()
            # 取出成交股數欄位值
            totalShare = record[2].replace(',', '').strip()
            # 取出成交金額欄位值
            totalTurnover = record[4].replace(',', '').strip()
            # 取出開盤價欄位值
            openPrice = record[5].replace(',', '').strip()
            # 取出最高價欄位值
            highestPrice = record[6].replace(',', '').strip()
            # 取出最低價欄位值
            lowestPrice = record[7].replace(',', '').strip()
            # 取出收盤價欄位值
            closePrice = record[8].replace(',', '').strip()
            afterHoursInfo = AfterHoursInfo(
                code=code,
                name=name,
                totalShare=totalShare,
                totalTurnover=totalTurnover,
                openPrice=openPrice,
                highestPrice=highestPrice,
                lowestPrice=lowestPrice,
                closePrice=closePrice
            )
            afterHoursInfos.append(afterHoursInfo)

    # 將每筆物件表達式輸出的字串以系統換行符號相接，讓每筆物件表達式各自獨立一行
    message = os.linesep.join([
        str(afterHoursInfo)
        for afterHoursInfo in afterHoursInfos
    ])
    loguru.logger.info('AFTERHOURSINFOS' + os.linesep + message)

In [21]:
def main():
    resp = requests.get(
        f'https://www.twse.com.tw/exchangeReport/MI_INDEX?' +
        f'response=json&' +
        f'type=ALLBUT0999' +
#         f'&date={datetime.date.today():%Y%m%d}'
        f'&date=20200424'
    )
    if resp.status_code != 200:
        loguru.logger.error('RESP: status code is not 200')
    loguru.logger.success('RESP: success')

    # 盤後資訊清單
    afterHoursInfos = []

    # 取出 JSON 內容
    body = resp.json()
    # 取出 stat 欄位
    stat = body['stat']
    # 如果 stat 不是 OK，代表查詢日期尚無資料
    if stat != 'OK':
        loguru.logger.error(f'RESP: body.stat error is {stat}.')
        return
    # 取出第 9 表格內容
    records = body['data9']
    # 依序取出每筆盤後資訊
    for record in records:
        # 取出代碼欄位值
        code = record[0].strip()
        # 符合股票代碼規則才處理
        if re.match(r'^[1-9][0-9][0-9][0-9]$', code) is not None:
            # 取出名稱欄位值
            name = record[1].strip()
            # 取出成交股數欄位值
            totalShare = record[2].replace(',', '').strip()
            # 取出成交金額欄位值
            totalTurnover = record[4].replace(',', '').strip()
            # 取出開盤價欄位值
            openPrice = record[5].replace(',', '').strip()
            # 取出最高價欄位值
            highestPrice = record[6].replace(',', '').strip()
            # 取出最低價欄位值
            lowestPrice = record[7].replace(',', '').strip()
            # 取出收盤價欄位值
            closePrice = record[8].replace(',', '').strip()
            afterHoursInfo = AfterHoursInfo(
                code=code,
                name=name,
                totalShare=totalShare,
                totalTurnover=totalTurnover,
                openPrice=openPrice,
                highestPrice=highestPrice,
                lowestPrice=lowestPrice,
                closePrice=closePrice
            )
            afterHoursInfos.append(afterHoursInfo)

    message = os.linesep.join([
        str(afterHoursInfo)
        for afterHoursInfo in afterHoursInfos
    ])
    loguru.logger.info('AFTERHOURSINFOS' + os.linesep + message)

if __name__ == '__main__':
    loguru.logger.add(
        f'{datetime.date.today():%Y%m%d}.log',
        rotation='1 day',
        retention='7 days',
        level='DEBUG'
    )
    main()

2020-04-29 16:52:19.030 | SUCCESS  | __main__:main:11 - RESP: success
2020-04-29 16:52:19.062 | INFO     | __main__:main:62 - AFTERHOURSINFOS
class AfterHoursInfo { Code=1101, Name=台泥, TotalShare=9369114, TotalTurnover=393496549, OpenPrice=42.15, HighestPrice=42.20, LowestPrice=41.70, ClosePrice=42.00 }
class AfterHoursInfo { Code=1102, Name=亞泥, TotalShare=4432295, TotalTurnover=184708323, OpenPrice=41.65, HighestPrice=41.90, LowestPrice=41.40, ClosePrice=41.70 }
class AfterHoursInfo { Code=1103, Name=嘉泥, TotalShare=349712, TotalTurnover=5627008, OpenPrice=16.15, HighestPrice=16.20, LowestPrice=15.95, ClosePrice=16.15 }
class AfterHoursInfo { Code=1104, Name=環泥, TotalShare=192096, TotalTurnover=3226751, OpenPrice=16.85, HighestPrice=16.90, LowestPrice=16.75, ClosePrice=16.85 }
class AfterHoursInfo { Code=1108, Name=幸福, TotalShare=57100, TotalTurnover=409242, OpenPrice=7.14, HighestPrice=7.20, LowestPrice=7.13, ClosePrice=7.20 }
class AfterHoursInfo { Code=1109, Name=信大, TotalShare=4918

## 取得個股每月各交易日盤後資訊

In [22]:
class AfterHoursDailyInfo:
    def __init__(
        self,
        code,
        date,
        totalShare,
        totalTurnover,
        openPrice,
        highestPrice,
        lowestPrice,
        closePrice):
        # 代碼
        self.Code = code
        # 日期
        # 國曆年轉為西元年
        parts = date.split('/')
        date = datetime.date(int(parts[0]) + 1911, int(parts[1]), int(parts[2]))
        self.Date = date
        # 成交股數
        self.TotalShare = int(totalShare)
        # 成交金額
        self.TotalTurnover = int(totalTurnover)
        # 開盤價
        self.OpenPrice = fractions.Fraction(openPrice)
        # 最高價
        self.HighestPrice = fractions.Fraction(highestPrice)
        # 最低價
        self.LowestPrice = fractions.Fraction(lowestPrice)
        # 收盤價
        self.ClosePrice = fractions.Fraction(closePrice)
    # 物件表達式
    def __repr__(self):
        return (
            f'class AfterHoursDailyInfo {{ '
            f'Code={self.Code}, '
            f'Date={self.Date:%Y-%m-%d}, '
            f'TotalShare={self.TotalShare}, '
            f'TotalTurnover={self.TotalTurnover}, '
            f'OpenPrice={float(self.OpenPrice):.2f}, '
            f'HighestPrice={float(self.HighestPrice):.2f}, '
            f'LowestPrice={float(self.LowestPrice):.2f}, '
            f'ClosePrice={float(self.ClosePrice):.2f} '
            f'}}'
        )

In [23]:
def main(year, month, code):
    date = f'{year}{month:02}01'
    resp = requests.get(
        f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?' +
        f'response=csv&date={date}&stockNo={code}')
    if resp.status_code != 200:
        loguru.logger.error('RESP: status code is not 200')
    loguru.logger.success('RESP: success')

    # 個股每月各交易日盤後資訊清單
    afterHoursDailyInfos = []
    # 取出 CSV 內容，並去除第一行及最後 5 行
    lines = io.StringIO(resp.text).readlines()
    lines = lines[1:-5]
    # 透過 CSV 讀取器載入
    reader = csv.DictReader(io.StringIO('\n'.join(lines)))
    # 依序取出每筆資料行
    for row in reader:
        # 取出日期欄位值
        date = row['日期'].strip()
        # 取出成交股數欄位值
        totalShare = row['成交股數'].replace(',', '').strip()
        # 取出成交金額欄位值
        totalTurnover = row['成交金額'].replace(',', '').strip()
        # 取出開盤價欄位值
        openPrice = row['開盤價']
        # 取出最高價欄位值
        highestPrice = row['最高價']
        # 取出最低價欄位值
        lowestPrice = row['最低價']
        # 取出收盤價欄位值
        closePrice = row['收盤價']
        afterHoursDailyInfo = AfterHoursDailyInfo(
            code=code,
            date=date,
            totalShare=totalShare,
            totalTurnover=totalTurnover,
            openPrice=openPrice,
            highestPrice=highestPrice,
            lowestPrice=lowestPrice,
            closePrice=closePrice
        )
        afterHoursDailyInfos.append(afterHoursDailyInfo)
    loguru.logger.info(afterHoursDailyInfos)

    # 將每筆物件表達式輸出的字串以系統換行符號相接，讓每筆物件表達式各自獨立一行
    message = os.linesep.join([
        str(afterHoursDailyInfo)
        for afterHoursDailyInfo in afterHoursDailyInfos
    ])
    loguru.logger.info('AFTERHOURSDAILYINFOS' + os.linesep + message)

if __name__ == '__main__':
    loguru.logger.add(
        f'{datetime.date.today():%Y%m%d}.log',
        rotation='1 day',
        retention='7 days',
        level='DEBUG'
    )
    # 傳入年、月及股票代碼
    main(2019, 4, '1101')

2020-04-29 16:52:20.561 | SUCCESS  | __main__:main:8 - RESP: success
2020-04-29 16:52:20.621 | INFO     | __main__:main:44 - [class AfterHoursDailyInfo { Code=1101, Date=2019-04-01, TotalShare=28261025, TotalTurnover=1171619600, OpenPrice=41.55, HighestPrice=41.80, LowestPrice=41.25, ClosePrice=41.35 }, class AfterHoursDailyInfo { Code=1101, Date=2019-04-02, TotalShare=23467533, TotalTurnover=974072590, OpenPrice=41.65, HighestPrice=41.70, LowestPrice=41.30, ClosePrice=41.40 }, class AfterHoursDailyInfo { Code=1101, Date=2019-04-03, TotalShare=19088018, TotalTurnover=794119481, OpenPrice=41.55, HighestPrice=41.75, LowestPrice=41.50, ClosePrice=41.70 }, class AfterHoursDailyInfo { Code=1101, Date=2019-04-08, TotalShare=28403967, TotalTurnover=1196467996, OpenPrice=41.95, HighestPrice=42.35, LowestPrice=41.95, ClosePrice=42.30 }, class AfterHoursDailyInfo { Code=1101, Date=2019-04-09, TotalShare=28568686, TotalTurnover=1204294549, OpenPrice=42.45, HighestPrice=42.55, LowestPrice=41.95, C

## 取得個股當日基本資訊 + 解析 PDF

In [24]:
# 因為 camelot 0.7.3 尚未修正 win32/win64 呼叫函式的格式問題（GitHub 上的
# master 分支已修正），故要覆寫 camelot 內呼叫 GhostScript API 的定義格式
camelot.ext.ghostscript._gsprint.c_stdstream_call_t = ctypes.WINFUNCTYPE(
    ctypes.c_int,
    camelot.ext.ghostscript._gsprint.gs_main_instance,
    ctypes.POINTER(ctypes.c_char),
    ctypes.c_int
)

def main(code):
    resp = requests.get(f'https://www.twse.com.tw/pdf/ch/{code}_ch.pdf')
    if resp.status_code != 200:
        loguru.logger.error('RESP: status code is not 200')
    loguru.logger.success('RESP: success')

    filename = f'{code}.pdf'
    filepath = f'{filename}'

    with open(filepath, 'wb') as f:
        f.write(resp.content)

    # 透過 camelot 辨識出 PDF 檔案內的表格
    tables = camelot.read_pdf(filepath)
    loguru.logger.info('DataFrame' + os.linesep + repr(tables[0].df))

    # 取出第 1 表格的 DataFrame 中的實收資本額
    paidin = tables[0].df[0][3]
    paidin.replace('新台幣', '').replace(',', '').strip()
    loguru.logger.info(f'實收資本額 {paidin}')

if __name__ == '__main__':
    loguru.logger.add(
        f'{datetime.date.today():%Y%m%d}.log',
        rotation='1 day',
        retention='7 days',
        level='DEBUG'
    )
    # 傳入股票代碼
    main('1101')

2020-04-29 16:52:21.665 | SUCCESS  | __main__:main:14 - RESP: success
2020-04-29 16:52:22.777 | INFO     | __main__:main:24 - DataFrame
                                                   0
0                                         台灣水泥股份有限公司
1                                                黃健強
2                        http://www.taiwancement.com
3                                 新台幣 56,656,192,040
4  有關水泥及水泥製品之生產及運銷 \n有關水泥原料及水泥製品原料之開採製造運銷及附屬礦石之開採...
5                                         1962/02/09
6                                               水泥工業
2020-04-29 16:52:22.779 | INFO     | __main__:main:29 - 實收資本額 新台幣 56,656,192,040
